In [15]:
from utils import *
from simanneal import Annealer
import random

root_dir = ""
label_count = 6

def populate_weights_map(path):
    weight_map = {}
    with open(path) as fr:
        for line in fr:
            tokens = line.strip().split()
            features = tokens[0].split(":")
            word = features[0].replace("Wi=","")
            label = features[-1].replace("Ti=","")
            weight_map[word+"_"+label] = float(tokens[1])
    return weight_map

class TagSolver(Annealer):
    
    def __init__(self, sentence, weights, state):
        self.sentence = sentence
        self.weights = weights
#         '': 8,
#  'B-LOC': 5,
#  'B-MISC': 6,
#  'B-ORG': 7,
#  'I-LOC': 4,
#  'I-MISC': 2,
#  'I-ORG': 0,
#  'I-PER': 3,
#  'O': 1}
        self.labels = ["O", "I-PER", "I-MISC", "B-MISC", "I-LOC", "B-LOC", "B-ORG", "I-ORG"]
        super().__init__(state)
    
    def move(self):
        for i in range(len(self.state)):
            self.state[i]= self.labels[random.randint(0, len(self.labels)-1)]
    
    def energy(self):
        obj_fn = 0
        for i,token in enumerate(self.sentence):
            wgt= self.weights.get(str(token)+"_"+self.state[i],0.0)
            obj_fn += exp(wgt)
        return -obj_fn
    
def get_prediction(weights, sentence):
    tagSolver = TagSolver( sentence, weights, ["O"]*len(sentence))
    state, energy = tagSolver.anneal()
    return (state, energy)

In [16]:
weights = populate_weights_map("./opti_data/weights")
sentences, labels, vocab, label_vocab = create_vocabulary("./opti_data/train")

EU NNP I-NP I-ORG
rejects VBZ I-VP O
German JJ I-NP I-MISC
call NN I-NP O
to TO I-VP O
boycott VB I-VP O
British JJ I-NP I-MISC
lamb NN I-NP O
. . O O
*****************
Peter NNP I-NP I-PER
Blackburn NNP I-NP I-PER
*****************
BRUSSELS NNP I-NP I-LOC
1996-08-22 CD I-NP O
*****************
The DT I-NP O
European NNP I-NP I-ORG
Commission NNP I-NP I-ORG
said VBD I-VP O
on IN I-PP O
Thursday NNP I-NP O
it PRP B-NP O
disagreed VBD I-VP O
with IN I-PP O
German JJ I-NP I-MISC
advice NN I-NP O
to TO I-PP O
consumers NNS I-NP O
to TO I-VP O
shun VB I-VP O
British JJ I-NP I-MISC
lamb NN I-NP O
until IN I-SBAR O
scientists NNS I-NP O
determine VBP I-VP O
whether IN I-SBAR O
mad JJ I-NP O
cow NN I-NP O
disease NN I-NP O
can MD I-VP O
be VB I-VP O
transmitted VBN I-VP O
to TO I-PP O
sheep NN I-NP O
. . O O
*****************
Germany NNP I-NP I-LOC
's POS B-NP O
representative NN I-NP O
to TO I-PP O
the DT I-NP O
European NNP I-NP I-ORG
Union NNP I-NP I-ORG
's POS B-NP O
veterinary JJ I-NP O
c

In [18]:
from math import exp

predictions = []
for sentence in sentences[:100]:
    state, _ = get_prediction(weights, sentence)
    predictions.append(state)

 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     2.50000        -30.26     5.00%     2.80%     0:00:03     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     2.50000         -8.39    53.40%     8.80%     0:00:02     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     2.50000         -1.37    89.00%    26.20%     0:00:02     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     2.50000        -61.91     3.80%     2.20%     0:00:07     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     2.50000        -55.84     3.20%     1.80%     0:00:07     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     2.50000        -38.76     5.40%     2.20%     0:00:07     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     2.50000        -37.78    24.20%    11.20%     0:00:06     0:00:00 Temperature 

     2.50000        -10.72    84.20%    25.60%     0:00:03     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     2.50000         -1.00   100.00%     0.00%     0:00:01     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     2.50000         -9.37    95.40%    16.20%     0:00:03     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     2.50000         -2.00    96.00%    21.80%     0:00:01     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     2.50000        -30.09    72.60%    30.20%     0:00:07     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     2.50000        -21.37    34.00%    14.00%     0:00:06     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     2.50000         -8.00    98.80%     8.60%     0:00:03     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     2.50000 

In [4]:
label_dict = {'': 8,
 'B-LOC': 5,
 'B-MISC': 6,
 'B-ORG': 7,
 'I-LOC': 4,
 'I-MISC': 2,
 'I-ORG': 0,
 'I-PER': 3,
 'O': 1}

In [5]:
from sklearn.metrics import accuracy_score 
import numpy as np
accs = []
for i, (pred, true) in enumerate(zip(predictions, labels)):
    acc = accuracy_score(true, [label_dict[x] for x in pred])
    accs.append((len(pred), acc))
    print(i, acc)
    print("predi:", pred)
    print("truth:", np.array(true))

0 0.4444444444444444
predi: ['B-ORG', 'I-MISC', 'I-MISC', 'O', 'O', 'O', 'O', 'I-PER', 'I-ORG']
truth: [0 1 2 1 1 1 2 1 1]
1 0.0
predi: ['I-MISC', 'O']
truth: [3 3]
2 0.0
predi: ['I-MISC', 'B-MISC']
truth: [4 1]
3 0.26666666666666666
predi: ['O', 'O', 'I-PER', 'I-LOC', 'I-PER', 'I-LOC', 'I-LOC', 'I-LOC', 'O', 'I-MISC', 'I-MISC', 'I-LOC', 'O', 'O', 'I-LOC', 'O', 'B-MISC', 'I-LOC', 'I-PER', 'B-LOC', 'I-ORG', 'B-LOC', 'I-PER', 'I-LOC', 'O', 'I-MISC', 'O', 'O', 'B-MISC', 'I-PER']
truth: [1 0 0 1 1 1 1 1 1 2 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
4 0.12903225806451613
predi: ['B-ORG', 'I-PER', 'I-PER', 'O', 'I-PER', 'O', 'B-MISC', 'I-MISC', 'I-LOC', 'O', 'B-MISC', 'B-ORG', 'I-ORG', 'O', 'I-ORG', 'O', 'I-LOC', 'I-PER', 'B-ORG', 'B-ORG', 'B-LOC', 'B-ORG', 'I-ORG', 'B-MISC', 'B-LOC', 'I-MISC', 'I-MISC', 'B-LOC', 'B-ORG', 'I-LOC', 'B-LOC']
truth: [4 1 1 1 1 0 0 1 1 1 3 3 1 1 1 1 1 1 1 1 1 1 1 4 1 1 1 1 1 1 1]
5 0.30303030303030304
predi: ['I-MISC', 'O', 'O', 'O', 'O', 'I-ORG', 'I-MISC', 'I-PE

In [14]:
# import pandas as pd

# df = pd.read_dict
accs.sort(key = lambda x: x[0])

for item in accs:
    print(str(item[0])+'\t'+str(item[1]*100))

1	0.0
1	100.0
1	100.0
1	100.0
1	100.0
1	100.0
1	100.0
1	100.0
1	100.0
2	0.0
2	0.0
2	0.0
2	0.0
2	0.0
2	0.0
2	0.0
2	0.0
2	50.0
3	33.33333333333333
5	80.0
5	100.0
5	100.0
7	71.42857142857143
7	71.42857142857143
7	71.42857142857143
7	100.0
7	100.0
8	0.0
8	0.0
8	0.0
8	12.5
8	25.0
8	87.5
8	87.5
8	100.0
8	100.0
9	11.11111111111111
9	11.11111111111111
9	11.11111111111111
9	22.22222222222222
9	44.44444444444444
9	55.55555555555556
9	66.66666666666666
9	77.77777777777779
9	88.88888888888889
9	100.0
9	100.0
10	20.0
10	20.0
10	100.0
10	100.0
10	100.0
11	18.181818181818183
11	18.181818181818183
11	27.27272727272727
11	100.0
12	16.666666666666664
13	7.6923076923076925
14	14.285714285714285
14	21.428571428571427
14	71.42857142857143
15	13.333333333333334
15	20.0
15	40.0
16	6.25
16	68.75
17	17.647058823529413
19	15.789473684210526
21	19.047619047619047
21	23.809523809523807
22	13.636363636363635
24	16.666666666666664
24	20.833333333333336
25	12.0
26	26.923076923076923
27	14.814814814814813
28	14.28571

In [ ]:
hist = {}

for l, a in accs:
    temp = hist.get(l, [])
    temp.append(a)
    hist[l] = temp

In [ ]:
predictions = create_objective_fn(final_params, inference_mode=True)

accs = []
for i, (pred, true) in enumerate(zip(predictions, labels)):
    acc = accuracy_score(true, pred)
    accs.append((len(pred), acc))
    print(i, acc)
    print("predi:", pred)
    print("truth:", np.array(true))


#Stochastic or Batch gradient descent?
#what about constraints
# is this even relevant to opti?
# why can't we use likelihood as obj function?
# no. of hidden states == label_count?

In [ ]:
global sentences, labels
sentences,labels, vocab, label_vocab = create_vocabulary("./data/files/train")
sentences = np.array(sentences)
labels = np.array(labels)
global label_count
label_count = len(label_vocab)
global vocab_size
vocab_size = len(vocab)
init_pi = [1/label_count]*label_count
init_pi = np.array(init_pi).reshape(label_count)

init_trans = [[1/label_count]*label_count for i in range(label_count)]
init_trans = np.array(init_trans).reshape(label_count*label_count)

init_em = [[1/vocab_size]*label_count for i in range(vocab_size)]
init_em = np.array(init_em).reshape(vocab_size*label_count)

init_params = np.concatenate((init_pi, init_trans, init_em))

# random initialization
#    init_params = np.random.uniform(0,1,(1 + len(vocab) + label_count)*label_count)

lagrangian_params = np.random.uniform(0,1,(2 + len(vocab) + label_count)*label_count)
init_full_params = np.concatenate((init_params, lagrangian_params))

objective_grad = grad(create_objective_fn)

final_params = init_full_params
final_params = pickle.load(open('results/final_params_100_100iters.pkl', 'rb'))

#    final_params = adam(objective_grad, final_params, step_size=0.01, callback=callback,
#                    num_iters=100)
pkl_name = 'results/final_params_' + ''.join(random.choice(string.ascii_letters) for _ in range(6)) + '.pkl'
pickle.dump(final_params, open(pkl_name, 'wb'))

print("********End of training *******")
#    print("optimal params")
#    print_params(final_params, label_count, vocab_size)
#    print("---------------------")
print("Total 